In [1]:
import numpy as np
import pandas as pd  # для работы с данными
import time  # для оценки времени
import torch  # для написания нейросети
from torch import nn
import re

# Шифр цезаря

In [2]:
CHARS = [chr(x) for x in range(ord('а'),ord('а')+32)]
CHAR_TO_INDEX_С = {w: i for i, w in enumerate(CHARS)}

In [3]:
def caesar_encode(in_text,chars,chars_index,k=2):
    return ''.join([chars[(chars_index[x]+k)%len(chars)] if x in chars else x for x in in_text])

def caesar_decode(in_text,chars,chars_index,k=2):
    return ''.join([chars[(chars_index[x]-k)%len(chars)] if x in chars else x for x in in_text])

In [4]:
caesar_encode("— Итак...— говорим мы.",CHARS,CHAR_TO_INDEX_С,k=2)

'— Ифвм...— ердртко оэ.'

In [5]:
caesar_decode('— Ифвм...— ердртко оэ.',CHARS,CHAR_TO_INDEX_С,k=2)

'— Итак...— говорим мы.'

In [6]:
INDEX_TO_CHAR = ['none'] + [w for w in CHARS+[' ']]
CHAR_TO_INDEX = {w: i for i, w in enumerate(INDEX_TO_CHAR)}

In [7]:
# Тот, кто ждёт. Рэй Брэдбери
text = '''Я живу в колодце. Я живу в колодце, как дымка. Как пар в каменной глотке. Я недвижим. Я ничего не делаю, только жду. Наверху я вижу холодные ночные и утренние звезды, вижу солнце. И пою иногда древние песни этого мира, песни времен его юности. Как я могу сказать, что я такое, когда и сам не знаю? Никак не могу. Я просто жду. Я — туман, я — лунный свет и память. Я печален и стар. Иногда я дождем падаю в колодец, и там, куда шлепаются мои быстрые капли, вода вздрагивает и покрывается узорчатой паутиной. Я жду в прохладной тиши, и наступит день, когда мне уже не придется ждать.
Сейчас утро. Я слышу мощный гром. Я чувствую вдалеке запах огня. Я слышу скрежет металла. Я жду. Я — слух.
Голоса. Далеко. — Порядок!
Один голос. Чужой голос. Неведомый язык, которого не знаю. Ни единого знакомого слова. Я слушаю.
— Выпускайте людей наружу.
Хруст песчинок.
— Где флаг?
— Здесь, сэр.
— Хорошо, хорошо.
Солнце стоит высоко в синем небе, его золотые лучи наполняют колодец, и я зависаю в нем — невидимое облачко в теплом свете.
Голоса.
— Именем правительства Земли объявляю эту планету Марсианской Территорией, равно поделенной между нациями.
Что они говорят? Я кружусь в лучах солнца, словно колесо, невидимый и неторопливый, золотистый и неутомимый.
— Что это тут?
— Колодец!
— Да ну!
— Ну-ка? Точно!
Приближается что-то теплое. Над зевом колодца склоняются три объекта, и моя прохлада поднимается к ним.
— Ух ты!
— Думаешь, там хорошая вода?
— Увидим.
— Эй, кто-нибудь, принесите склянку и шпагат.
— Я принесу.
Звук бегущих шагов. Удаляющихся. Теперь приближающихся.
— Вот.
Я жду.
— Опускаем. Легонько.
Склянка, поблескивая, медленно опускается на шпагате. Вода подернулась мелкой рябью, когда склянка коснулась ее и наполнилась. Я поднимаюсь в теплом воздухе к жерлу колодца.
— Вот. Хотите отведать этой водицы, Риджент?
— Давайте.
— Что за чудесный колодец! Взгляните, как он устроен.
Сколько ему лет, как вы думаете?
— Бог знает. Когда мы вчера сели в том, другом, городе, Смит сказал, что уже десять тысяч лет как на Марсе нет жизни.
— Подумать только!
— Ну что, Риджент, как водичка?
— Чистый хрусталь. Выпей стаканчик.
Плеск воды на солнцепеке. Теперь я плыву буроватой пылью на легком ветерке.
— В чем дело, Джонс?
— Не знаю. Ужасно заболела голова. Ни с того ни с сего.
— Ты еще не пил эту воду?
— Нет. Дело не в ней. Я просто склонился над колодцем, и голова вдруг стала раскалываться. Но сейчас уже лучше.
Теперь я знаю, кто я.
Мое имя — Стивен Леонард Джонс, мне двадцать пять лет, и я только что прилетел в ракете с планеты под названием Земля. А теперь стою с моими добрыми друзьями Риджентом и Шоу возле старого колодца на планете Марс.
Я смотрю на свои золотистые пальцы, загорелые и крепкие, оглядываю свои длинные ноги и серебристую форму, оглядываю моих друзей.
— Что случилось, Джонс? — спрашивают они.
— Ничего,— говорю я, глядя на них,— ровным счетом ничего.
До чего же вкусна пища! Я не ел уже десять тысяч лет. Пища нежно ласкает мой язык, а вино, которым я запиваю ее, согревает. Я слушаю голоса. Я составляю слова, которых не понимаю и все же как-то понимаю. Я пробую на вкус воздух.
— В чем дело, Джонс?
Я склоняю набок эту свою голову и опускаю руки, которые держат серебристую емкость с пищей. Мне доступны все ощущения.
— О чем вы? — спрашивает этот мой голос. Это моя новая штуковина.
— Ты странно дышишь, с покашливанием,— говорит второй человек.
— Может, начинается легкая простуда,— заявляю я.
— Потом пойдешь к доктору, обследуешься.
Я киваю головой, и кивать приятно. Приятно сделать хоть что-нибудь спустя десять тысяч лет. Приятно подышать воздухом и почувствовать, как солнце прогревает плоть, все глубже и глубже. Приятно ощутить твердую кость, тонкий скелет в согревшейся плоти, приятно слышать звуки гораздо яснее, гораздо ближе, чем там, в каменных глубинах колодца. Я сижу как зачарованный.
— Очнись, Джонс. Вставай. Надо идти.
— Да,— отвечаю я, завороженный тем, как слово рождается на языке, как оно медленно и красиво падает в воздух.
Я иду. И идти приятно. Я выпрямляюсь и смотрю на землю. Она далеко от глаз и от головы. Как будто живешь на прекрасном утесе.
Риджент стоит у каменного колодца, глядит в него. Остальные, бормоча что-то, ушли к серебристому кораблю, из которого они появились.
Я чувствую свои пальцы и ощущаю улыбку у себя на губах.
— Он глубокий,— говорю я.
— Да.
— Он называется Колодец Душ.
Риджент поднимает голову и смотрит на меня. — Откуда ты это знаешь?
— А что, разве не похоже?
— Я никогда прежде не слыхал о Колодце Душ.
— Это место, где все ждущие, те, кто раньше имел плоть, ждут и ждут без конца,— говорю я, касаясь его руки.
Песок — огонь, а корабль — серебряный огонь в дневном зное. И чувствовать зной приятно. Звук моих шагов по твердому песку. Я слушаю. Шум ветра и гул солнца, жгущего долины. Я вдыхаю запах кипящей в полдень ракеты. Я стою под люком.
— Где Риджент? — спрашивает кто-то.
— Видел его у колодца,— отвечаю я.
Один из них бежит к колодцу.
Я начинаю дрожать. Тихая трясучая дрожь, затаившаяся глубоко внутри, становится все сильнее. И я впервые слышу его, как будто он хоронился вместе со мной в колодце: глубоко внутри кричит голос, тонкий и испуганный. Голос кричит: «Отпусти меня, отпусти меня»,— и такое чувство, будто что-то рвется на волю; хлопают двери в лабиринте, что-то бежит по темным коридорам и переходам, слышатся крики.
— Риджент в колодце!
Люди бегут. Я бегу с ними, но мне дурно, и дрожь буквально неистовствует.
— Наверное, он упал. Джонс, ты был здесь с ним. Ты видел? Джонс? Ну, говори же, парень!
— В чем дело, Джонс?
Я падаю на колени. Как же сильно меня трясет!
— Он болен. Эй, помогите-ка поднять его.
— Это солнце.
— Нет,— бормочу я,— не солнце.
Они укладывают меня на спину; судороги похожи на подземные толчки, и голос, спрятанный глубоко во мне, кричит: «Это Джонс, это я, это не он, это не он, не верьте ему, выпустите меня, выпустите меня!» И я смотрю вверх на склоненные фигуры и моргаю. Они касаются моих запястий.
— У него сердце частит.
Я закрываю глаза. Крики стихают. Дрожь прекращается. Освобожденный, я поднимаюсь, будто в прохладном колодце.
— Он мертв,— говорит кто-то.
— Джонс умер.
— Отчего?
— Похоже, от шока.
— От какого шока? — спрашиваю я. Меня зовут Сешшнз, мои губы с трудом шевелятся, и я — капитан этих людей. Я стою среди них и смотрю вниз, на тело, которое лежит и остывает на песке. Я хватаю себя за голову обеими руками.
— Капитан!
— Ничего! — кричу я.— Просто головная боль. Все будет в порядке.
«Ну, ну,— шепчу я.— Теперь все хорошо».
— Нам лучше уйти в тень, сэр.
— Да,— говорю я, глядя сверху на Джонса.— Нам не надо было прилетать: Марс не хочет нас.
Мы несем тело к кораблю, а глубоко внутри меня кричит новый голос, требуя свободы.
«Помогите, помогите...— звучит где-то далеко, во влажной плоти.— Помогите, помогите...» — эхом разносятся слова мольбы, словно красные призраки.
На этот раз дрожь начинается гораздо раньше. Я не могу управлять собой, как прежде.
— Капитан, вы бы лучше перешли в тень. У вас неважный вид, сэр.
— Да,— говорю я.— Помогите,— говорю я.
— Что, сэр?
— Я ничего не говорил.
— Вы сказали «помогите», сэр.
— Я сказал? Мэтьюс, я сказал?
Тело укладывают в тени ракеты, и голос кричит в глубине затопленных водой катакомб из костей и багровых потоков. Мои руки сводит судорогой. Мой разинутый рот иссох. Мои ноздри раздуваются. Мои глаза закатываются. «Помогите, помогите, о, помогите! Не надо, не надо, выпустите меня, выпустите меня!»
— Не надо,— говорю я.
— Что, сэр?
— Ничего,— говорю я.— Я должен освободиться,— говорю я, зажимая ладонью рот.
— Как так, сэр? — кричит Мэтьюс.
— Все — в ракету! — воплю я.— Возвращайтесь на Землю!
У меня в руках пистолет. Я поднимаю его.
Хлопок. Бегущие тени. Крик обрывается. Слышится свист, с которым несешься сквозь космос.
Как приятно умереть через десять тысяч лет. Как приятно почувствовать эту желанную прохладу, эту расслабленность. Как славно ощущать себя рукой в перчатке, которая вытягивается и становится дивно холодной на горячем песке. О, этот покой, эта благость сгущающейся тьмы смерти. Но ее нельзя продлить.
Треск. Щелчок.
— Боже всемогущий, он покончил с собой! — кричу я и открываю глаза. И вижу капитана. Он лежит, привалившись к ракете; голова раскроена пулей, глаза навыкате, язык высовывается меж белых зубов. Из головы хлещет кровь. Я склоняюсь над ним и касаюсь его.
— Глупец! — говорю я.— Зачем он это сделал?
Парней охватывает ужас. Они стоят над двумя мертвецами и, повернув головы, смотрят на марсианские пески и далекий колодец, где под толщей воды лежит Риджент.
С их сухих губ срывается хрип, они хнычут, будто дети в кошмарном сне.
Они поворачиваются ко мне.
После долгого молчания один из них говорит:
— Значит, теперь капитан ты, Мэтьюс.
— Знаю,— медленно говорю я.
— Нас осталось только шестеро.
— Боже, как быстро все произошло!
— Я не хочу оставаться тут. Давайте сматываться!
Парни галдят. Я подхожу и дотрагиваюсь до каждого из них. Я так уверен в себе, что хочется петь.
— Слушайте,— говорю я и касаюсь их локтей, плеч или кистей.
Мы умолкаем.
Мы — единое целое.
«Нет, нет, нет, нет, нет, нет!» — кричат внутренние голоса, они глубоко, в темницах наших тел.
Мы переглядываемся. Мы — Самуэль Мэтьюс, Чарлз Ивэнс, Форрест Коул, Рэймонд Мозес, Вильям Сполдинг и Джон Саммерс, и мы молчим. Мы только смотрим друг на друга, на наши бледные лица и трясущиеся руки.
Мы поворачиваемся как один и глядим в колодец.
— Итак...— говорим мы.
«Нет, нет!» — кричат шесть голосов, запрятанных, придавленных и погребенных навеки.
Наши ноги ступают по песку, и чувство такое, словно громадная двенадцатипалая ладонь отчаянно цепляется за теплое морское дно.
Мы склоняемся над колодцем, пока не теряем равновесие и не валимся по очереди в зев, летя сквозь прохладный мрак вниз, в холодные воды.
'''.lower()

check_text = '''Солнце садится. Звезды кружатся в ночном небе. Там, вдалеке, мерцает огонек. Приближается другая ракета, прочерчивая космос красным пунктиром.
Я живу в колодце. Я живу в колодце, как дымка. Как пар в каменной глотке. Наверху я вижу холодные ночные и утренние звезды, вижу солнце. И пою иногда древние песни этого мира, песни времен его юности. Как я могу сказать, что я такое, когда и сам не знаю? Никак не могу.
Я просто жду.'''.lower()

In [8]:
def text_proc(text):
    phras = [x.strip() for x in re.split(r';|\.|\!|\?',text.replace('\n','')) if len(x)>0]
    phras = [re.sub(r'[^\w\s\d]','',x).strip() for x in phras]
    phras = [re.sub(r'\s+',' ',x).strip() for x in phras]
    return phras

In [9]:
text = text_proc(text)
text[:5]

['я живу в колодце',
 'я живу в колодце как дымка',
 'как пар в каменной глотке',
 'я недвижим',
 'я ничего не делаю только жду']

In [10]:
K = 2
check_text = text_proc(check_text)
check_text = [ caesar_encode(x,CHARS,CHAR_TO_INDEX_С,k=K) for x in check_text]
check_text[:5]

['урнпшз увжкфуб',
 'йдзйжэ мтхивфуб д прщпро пзгз',
 'фво джвнзмз озтшвзф рерпзм',
 'сткгнкивзфуб жтхевб твмзфв стрщзтщкдвб мруору мтвупэо схпмфктро',
 'б икдх д мрнржшз']

In [11]:
#pd.DataFrame(np.array([len(x) for x in text])).hist(bins=50)

In [12]:
MAX_LEN = 20  # мы хотим ограничить максимальную длину ввода
K=2
X = torch.zeros((len(text), MAX_LEN), dtype=int)  # создаём пустой вектор для текста, чтобы класть в него индексы токенов
Y = torch.zeros((len(text), MAX_LEN), dtype=int)

for i in range(len(text)):  # для каждого предложения
    
    #сначала шифруем фразу, затем делаем таргет
    caesar = caesar_encode(text[i],CHARS,CHAR_TO_INDEX_С,k=K)
    for j, w in enumerate(caesar):  # для каждого токена
        if j >= MAX_LEN:
            break
        X[i, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])
        
    for j, w in enumerate(text[i]):  # для каждого токена
        if j >= MAX_LEN:
            break
        Y[i, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])
 

In [13]:
print(f'x train shape {X.shape}, y train shape {Y.shape}, {len(INDEX_TO_CHAR)}')

x train shape torch.Size([272, 20]), y train shape torch.Size([272, 20]), 34


In [14]:
class Network(torch.nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.embedding = torch.nn.Embedding(34, 34)
        self.rnn = torch.nn.RNN(34, 128)
        self.out = torch.nn.Linear(128, 34)

    def forward(self, sentences, state=None):
        x = self.embedding(sentences)
        x,s = self.rnn(x)
        return self.out(x)

In [15]:
model = Network()

In [16]:
criterion = torch.nn.CrossEntropyLoss()  # типичный лосс многоклассовой классификации
optimizer = torch.optim.SGD(model.parameters(), lr=.05)

In [18]:
batch_size = 32

for ep in range(300):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    for i in range(int(len(X) / batch_size)):
        X_batch = X[i * batch_size:(i + 1) * batch_size]
        Y_batch = Y[i * batch_size:(i + 1) * batch_size].flatten()
        optimizer.zero_grad()
        answers = model.forward(X_batch)
        answers = answers.view(-1, len(INDEX_TO_CHAR))
        loss = criterion(answers, Y_batch)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1
    if ep%20==0:
        print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))


Epoch 0. Time: 0.061, Train loss: 3.324
Epoch 20. Time: 0.045, Train loss: 0.484
Epoch 40. Time: 0.048, Train loss: 0.200
Epoch 60. Time: 0.045, Train loss: 0.105
Epoch 80. Time: 0.046, Train loss: 0.065
Epoch 100. Time: 0.045, Train loss: 0.045
Epoch 120. Time: 0.044, Train loss: 0.034
Epoch 140. Time: 0.048, Train loss: 0.027
Epoch 160. Time: 0.048, Train loss: 0.022
Epoch 180. Time: 0.049, Train loss: 0.018
Epoch 200. Time: 0.042, Train loss: 0.016
Epoch 220. Time: 0.043, Train loss: 0.014
Epoch 240. Time: 0.046, Train loss: 0.012
Epoch 260. Time: 0.046, Train loss: 0.011
Epoch 280. Time: 0.047, Train loss: 0.010


In [19]:
def decode_sentence(word):
    sentence = list(word)
    sentence = [CHAR_TO_INDEX.get(s, CHAR_TO_INDEX['none']) for s in sentence]
    answers = model.forward(torch.tensor(sentence))
    probas, indices = answers.topk(1)
    return ''.join([INDEX_TO_CHAR[ind.item()] for ind in indices.flatten()])

In [20]:
check_text

['урнпшз увжкфуб',
 'йдзйжэ мтхивфуб д прщпро пзгз',
 'фво джвнзмз озтшвзф рерпзм',
 'сткгнкивзфуб жтхевб твмзфв стрщзтщкдвб мруору мтвупэо схпмфктро',
 'б икдх д мрнржшз',
 'б икдх д мрнржшз мвм жэомв',
 'мвм свт д мвозппрл енрфмз',
 'пвдзтчх б дких чрнржпэз прщпэз к хфтзппкз йдзйжэ дких урнпшз',
 'к сра кпрежв жтздпкз сзупк яфрер октв сзупк дтзозп зер апруфк',
 'мвм б орех умвйвфю щфр б фвмрз мрежв к уво пз йпва',
 'пкмвм пз орех',
 'б струфр ижх']

In [21]:
result = []
for s in check_text:
    result.append(decode_sentence(s))

In [22]:
result

['солнце садится',
 'звезды кружатся в ночном небе',
 'там вдалеке мерцает огонек',
 'приближается другая ракета прочерчивая космос красным пунктиром',
 'я живу в колодце',
 'я живу в колодце как дымка',
 'как пар в каменной глотке',
 'наверху я вижу холодные ночные и утренние звезды вижу солнце',
 'и пою иногда древние песни этого мира песни времен его юности',
 'как я могу сказать что я такое когда и сам не знаю',
 'никак не могу',
 'я просто жду']

Алгоритм запомнил соответствие символов

# Практика из лекции

In [7]:
import re
import string

In [8]:
df = pd.read_csv(r"./simpsons_script_lines.csv",low_memory=False)
df.head(5)

,id,episode_id,number,raw_text,timestamp_in_ms,speaking_line,character_id,location_id,raw_character_text,raw_location_text,spoken_words,normalized_text,word_count
0,9549,32,209,"Miss Hoover: No, actually, it was a little of ...",848000,true,464,3.0,Miss Hoover,Springfield Elementary School,"No, actually, it was a little of both. Sometim...",no actually it was a little of both sometimes ...,31
1,9550,32,210,Lisa Simpson: (NEAR TEARS) Where's Mr. Bergstrom?,856000,true,9,3.0,Lisa Simpson,Springfield Elementary School,Where's Mr. Bergstrom?,wheres mr bergstrom,3
2,9551,32,211,Miss Hoover: I don't know. Although I'd sure l...,856000,true,464,3.0,Miss Hoover,Springfield Elementary School,I don't know. Although I'd sure like to talk t...,i dont know although id sure like to talk to h...,22
3,9552,32,212,Lisa Simpson: That life is worth living.,864000,true,9,3.0,Lisa Simpson,Springfield Elementary School,That life is worth living.,that life is worth living,5
4,9553,32,213,Edna Krabappel-Flanders: The polls will be ope...,864000,true,40,3.0,Edna Krabappel-Flanders,Springfield Elementary School,The polls will be open from now until the end ...,the polls will be open from now until the end ...,33


In [9]:
phrases = df['normalized_text'].tolist()  # колонка с предобработанными текстами
phrases[:10]

['no actually it was a little of both sometimes when a disease is in all the magazines and all the news shows its only natural that you think you have it',
 'wheres mr bergstrom',
 'i dont know although id sure like to talk to him he didnt touch my lesson plan what did he teach you',
 'that life is worth living',
 'the polls will be open from now until the end of recess now just in case any of you have decided to put any thought into this well have our final statements martin',
 'i dont think theres anything left to say',
 'bart',
 'victory party under the slide',
 nan,
 'mr bergstrom mr bergstrom']

In [10]:
CHARS = string.ascii_lowercase  + " " #+ string.digits
n_chars = len(CHARS)

In [11]:
#немного почистим датасет, уберём пустые записи, записи с символами, отличными от нашего словаря
ds_n = len(phrases)
phrases = [ x for x in phrases if isinstance(x,str) and len(x.strip()) >1]
phrases = [ x for x in phrases if not re.search(fr"[^{CHARS}]",x)]
print(f'shape {ds_n} to {len(phrases)}')

shape 158271 to 117006


In [12]:
INDEX_TO_CHAR = ['none'] + [w for w in CHARS] 
CHAR_TO_INDEX = {w: i for i, w in enumerate(INDEX_TO_CHAR)}  # словарь токен-индекс
n_chars +=1
n_chars

28

In [14]:
text = [[c for c in ph] for ph in phrases if type(ph) is str]

In [23]:
MAX_LEN = 50  # мы хотим ограничить максимальную длину ввода
X = torch.zeros((len(phrases), MAX_LEN), dtype=int)  # создаём пустой вектор для текста, чтобы класть в него индексы токенов
for i in range(len(phrases)):  # для каждого предложения
    for j, w in enumerate(phrases[i]):  # для каждого токена
        if j >= MAX_LEN:
            break
        X[i, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])

In [41]:
X.shape

torch.Size([117006, 50])

In [431]:
#создаём RNN class
class customRNN(nn.Module):
    
    def __init__(self, in_n, hidden_n,batch_first=False):
        super(customRNN, self).__init__()
        
        self.input_size = in_n
        self.hidden_size = hidden_n
        self.batch_first = batch_first
        
        self._in = nn.Linear(in_n, hidden_n)
        self.hidden = nn.Linear(hidden_n, hidden_n)
       
    def forward(self, sequences,h_0=None):
        
        if self.batch_first:
            sequences = sequences.transpose(0, 1)
        
        seq_len, batch_size, _ = sequences.shape
        
        if h_0 is None:
            h_0 = self.initHidden(batch_size)  
        
        h_prev = h_0
        output = []
        
        for i in range(seq_len):
            
            x = self._in(sequences[i])
            h_current = self.hidden(h_prev)
            h_current = torch.tanh(x+h_current)
            output.append(h_current)
            h_prev = h_current
        
        output = torch.stack(output)
        
        if self.batch_first:
            output = output.transpose(0, 1)
            
        return output, h_current

    def initHidden(self,batch_size):
        return torch.zeros(batch_size, self.hidden_size)

In [432]:
class fModel(nn.Module):
    def __init__(self, in_n, n_embedding,hidden_n,out_n):
        super(fModel, self).__init__()
        
        self.hidden_size = hidden_n
        
        self.embedding = torch.nn.Embedding(n_chars, n_embedding)
        self.rnn = customRNN(n_embedding, hidden_n, batch_first=True)
        self.out = nn.Linear(hidden_n, out_n)
        
    def forward(self, x, h=None):
        output = self.embedding(x) 
        if h is None:
            h = self.rnn.initHidden(output.shape[0]).requires_grad_()
        output, h = self.rnn(output, h.detach()) 
        return self.out(output),h

In [433]:
n_hidden = 256
n_embedding = 32
model = fModel(n_chars, n_embedding, n_hidden, n_chars)

In [434]:
lr = 0.001
loss = torch.nn.CrossEntropyLoss()  # типичный лосс многоклассовой классификации
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [435]:
#тренируем модель
batch_size = 32
ep_n = 10

#hidden = model.init_rnn()#.to(device)
model.train()
for ep in range(ep_n):
    start = time.time()
    train_loss = 0.
    train_passed = 0
    
    H = None
    for i in range(int(len(phrases) / batch_size)):
        
        batch = X[i * batch_size:(i + 1) * batch_size]
        X_batch = batch[:, :-1]
        Y_batch = batch[:, 1:].flatten()
        
        optimizer.zero_grad()
        output,H = model.forward(X_batch,H)
        output = output.view(-1, n_chars)
        l = loss(output, Y_batch)
        l.backward()
        optimizer.step()
        
        train_loss += l.item()
        train_passed += 1
        
        #if i%1000==0:
        #    print("Iter {}. Time: {:.3f}, Train loss: {:.3f}".format(i, time.time() - start, train_loss / train_passed))
    if ep%1==0:
        print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))
print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))

Epoch 0. Time: 99.402, Train loss: 1.159
Epoch 1. Time: 102.877, Train loss: 1.021
Epoch 2. Time: 101.128, Train loss: 0.992
Epoch 3. Time: 101.887, Train loss: 0.977
Epoch 4. Time: 101.886, Train loss: 0.967
Epoch 5. Time: 96.772, Train loss: 0.961
Epoch 6. Time: 98.829, Train loss: 0.956
Epoch 7. Time: 100.998, Train loss: 0.953
Epoch 8. Time: 97.609, Train loss: 0.950
Epoch 9. Time: 99.240, Train loss: 0.948
Epoch 9. Time: 99.240, Train loss: 0.948


In [436]:
#функция случайного выбора из топ символов
def choice_last_symb(indices):
    inds = indices.flatten()
    c = np.random.choice(inds.shape[0])
    return inds[c]

In [437]:
def generate_sentence(word,gen_len = 20,top_k = 1):
    
    model.eval()

    #прогрев
    sentence = list(word)
    sentence = [CHAR_TO_INDEX.get(s, 0) for s in sentence]
    _,h = model.forward(torch.tensor(sentence).view(1,-1))

    last_symb = torch.tensor(sentence[-1])
    
    #генерация
    predicted = ""
    for i in range(gen_len):
        output,h = model.forward(last_symb.view(1,-1),h)
        #get_last_symb(output.squeeze())
        probas, indices = output.topk(top_k)
        last_symb = choice_last_symb(indices)
        predicted +=INDEX_TO_CHAR[last_symb.item()]

    return predicted

In [440]:
generate_sentence('start ',20,1)

'the boy i want to se'

In [439]:
generate_sentence('sweetie maybe its time you retired from ',20)

'enjoy the boy i want'

In [441]:
generate_sentence('thats my boy well if you believe in me then im not going to ')

'pop money i want to '

In [447]:
generate_sentence('what a ',20,3)

'englack is so morroh'

In [449]:
generate_sentence('well homer too theyre so muc',30,2)

'home of that i woul shes a big'

In [444]:
generate_sentence('homer homer ',30,1)

'love the boy i want to see you'